# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:

import collections

import helper
import numpy as np
import project_tests as tests


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU,SimpleRNN, Dropout,Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam,RMSprop
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17154525601934652772
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 3868721152
locality {
  bus_id: 1
}
incarnation: 13009492346395363001
physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 3868721152
locality {
  bus_id: 1
}
incarnation: 12038579299471160549
physical_device_desc: "device: 1, name: GRID K520, pci bus id: 0000:00:04.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 3868721152
locality {
  bus_id: 1
}
incarnation: 2565741773233540505
physical_device_desc: "device: 2, name: GRID K520, pci bus id: 0000:00:05.0"
, name: "/gpu:3"
device_type: "GPU"
memory_limit: 3868721152
locality {
  bus_id: 1
}
incarnation: 14497056270421703814
physical_device_desc: "device: 3, name: GRID K520, pci bus id: 0000:00:06.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenizer.word_index['<PAD>'] =  0
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21, '<PAD>': 0}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return  pad_sequences(x, maxlen=length,padding='post', truncating='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
from sklearn.model_selection import train_test_split

def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)


max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)


#split data for testing. use 10% for testing
preproc_english_sentences,preproc_english_sentences_test, preproc_french_sentences, preproc_french_sentences_test =\
train_test_split(preproc_english_sentences, preproc_french_sentences, test_size=0.10, random_state=50)

#reshape testing english sentences
english_test = pad(preproc_english_sentences_test, max_french_sequence_length)
english_test = english_test.reshape((-1, preproc_french_sentences.shape[-2], 1))


print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 200
French vocabulary size: 345


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    
    #already <PAD> token to tokenizer function
    #index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1) if prediction !=0])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [11]:
def ids_to_text(ids, tokenizer):
    """
    Convert sentence of ints to sentence of text
    
    paramter:
        ids:         array of ints representings ids of words
        tokenizer:   tokenizer to covert word id to text
    
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    
    return ' '.join([index_to_words[id] for id in ids if id != 0])
        

def make_prediction(count,model):    
    """ 
    Make english to french translation prediction for given count randomly.
    """
    
    test_count = 1
    for id in np.random.randint(len(preproc_english_sentences_test), size=count):
        print("Test {}".format(test_count))
        #print english sentence
        print("\tEnglish sentence: \t{}".format(ids_to_text(preproc_english_sentences_test[id],english_tokenizer)))
        
        #get the input shape of the model and reshape the test english sentences
        input_shape = model.input_shape
        
        #get the sentence length of input and apply padding if necessary to the english sentences
        if preproc_english_sentences_test.shape[1] != input_shape[1]:
            english_test_x =  pad(preproc_english_sentences_test, max_french_sequence_length)
        else:
            english_test_x = preproc_english_sentences_test
        
        #reshape the english_test input sentences
        temp_list = list(input_shape)
        temp_list[0]=1
        input_shape = tuple(temp_list)
        temp = english_test_x[id].reshape(input_shape)
        
        #make prediction
        print("\tPrediction:     \t{}".format(logits_to_text(model.predict(temp)[0], french_tokenizer)))
        
        #print actaul french sentence
        print("\tFrench sentence: \t{}".format(ids_to_text(preproc_french_sentences_test[id].flatten(),french_tokenizer)))
        
        print("-" * 20 )
        test_count +=1

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    model = Sequential() 
    model.add(LSTM(500, return_sequences=True, input_shape=input_shape[1:]))
    model.add(Dense(french_vocab_size, activation='softmax'))
    rmsprop =  RMSprop(lr=0.01)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=rmsprop,
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

In [13]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(simple_rnn_model.summary())
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 21, 500)           1004000   
_________________________________________________________________
dense_2 (Dense)              (None, 21, 345)           172845    
Total params: 1,176,845.0
Trainable params: 1,176,845
Non-trainable params: 0.0
_________________________________________________________________
None
Train on 99259 samples, validate on 24815 samples
Epoch 1/10
99259/99259 [==============================] - 44s - loss: 1.8949 - acc: 0.5368 - val_loss: 1.3127 - val_acc: 0.6171
Epoch 2/10
99259/99259 [==============================] - 42s - loss: 1.2722 - acc: 0.6218 - val_loss: 1.1895 - val_acc: 0.6314
Epoch 3/10
99259/99259 [==============================] - 42s - loss: 1.1707 - acc: 0.6405 - val_loss: 1.1474 - val_acc: 0.6458
Epoch 4/10
99259/99259 [==============================] - 42s - loss: 1.1151 - acc: 0

In [41]:
#evaluate performance on the test dataset

# Reshaping the test input
tmp_x_test = pad(preproc_english_sentences_test, max_french_sequence_length)
tmp_x_test = tmp_x_test.reshape((-1, preproc_french_sentences.shape[-2], 1))
result = simple_rnn_model.evaluate(tmp_x_test,preproc_french_sentences_test, verbose=0)
print("Simple model accuracy on test dataset: {0:.4f}".format(result[1]) )

Simple model accuracy on test dataset: 0.6659


In [15]:
#make 5 predictions
make_prediction(5, simple_rnn_model)

Test 1
	English sentence: 	france is warm during may and it is usually relaxing in february
	Prediction:     	la est sec en mois de il est il est en février
	French sentence: 	france est chaud au mois de mai et il est relaxant habituellement en février
--------------------
Test 2
	English sentence: 	paris is wonderful during autumn and it is never nice in april
	Prediction:     	paris inde est en mois de il est il agréable agréable en en
	French sentence: 	paris est merveilleux au cours de l' automne et il est jamais agréable en avril
--------------------
Test 3
	English sentence: 	china is never hot during march but it is sometimes chilly in january
	Prediction:     	la est jamais jamais en en mais mais mais parfois parfois parfois en
	French sentence: 	chine est jamais chaud au mois de mars mais il est parfois froid en janvier
--------------------
Test 4
	English sentence: 	he liked that big blue automobile
	Prediction:     	il aimait une petite voiture vert
	French sentence: 	il aim

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [16]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential() 
    model.add(Embedding(french_vocab_size, 300, input_length=input_shape[1]))
    model.add(LSTM(500,return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    rmsprop =  RMSprop(lr=0.01)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=rmsprop,
                  metrics=['accuracy']) 
   
    return model
tests.test_embed_model(embed_model)

In [17]:
# TODO: Reshape the input
tmp_x2 = pad(preproc_english_sentences, max_french_sequence_length)

# TODO: Train the neural network
embed_model = embed_model(
    tmp_x2.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(embed_model.summary())
embed_model.fit(tmp_x2, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 300)           103500    
_________________________________________________________________
lstm_4 (LSTM)                (None, 21, 500)           1602000   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           172845    
Total params: 1,878,345.0
Trainable params: 1,878,345
Non-trainable params: 0.0
_________________________________________________________________
None
Train on 99259 samples, validate on 24815 samples
Epoch 1/10
99259/99259 [==============================] - 53s - loss: 2.3008 - acc: 0.5384 - val_loss: 1.1151 - val_acc: 0.7004
Epoch 2/10
99259/99259 [==============================] - 51s - loss: 0.6229 - acc: 0.8134 - val_loss: 0.4334 - val_acc: 0.8561
Epoch 3/10
99259/99259 [==============================] - 51s - loss: 0.3047 - ac

In [18]:
#evaluate performance on the test dataset

# Reshaping the test input
tmp_x2_test = pad(preproc_english_sentences_test, max_french_sequence_length)

result = embed_model.evaluate(tmp_x2_test,preproc_french_sentences_test, verbose=0)
print("Embed model accuracy on test dataset: {0:.4f}".format(result[1]) )

Embed model accuracy on test dataset: 0.9315


In [19]:
#make 5 predictions
make_prediction(5, embed_model)

Test 1
	English sentence: 	california is hot during february but it is snowy in march
	Prediction:     	californie est chaud en février mais il est neigeux en mars
	French sentence: 	californie est chaud en février mais il est neigeux en mars
--------------------
Test 2
	English sentence: 	our least liked fruit is the strawberry but my least liked is the grapefruit
	Prediction:     	notre fruit aimé des est la fraise mais mon moins aimé est la pamplemousse
	French sentence: 	notre moins aimé fruit est la fraise mais mon moins aimé est le pamplemousse
--------------------
Test 3
	English sentence: 	the lime is her least favorite fruit but the pear is their least favorite
	Prediction:     	les chaux est son fruit préféré moins mais la poire est leur moins préférée
	French sentence: 	la chaux est son fruit préféré moins mais la poire est leur moins préférée
--------------------
Test 4
	English sentence: 	india is never relaxing during may and it is never wet in november
	Prediction:     	

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [20]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    model = Sequential() 
    #model.add(Bidirectional(LSTM(output_sequence_length, return_sequences=True), input_shape=input_shape[1:]))
    model.add(Bidirectional(LSTM(500, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    rmsprop =  RMSprop(lr=0.01)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=rmsprop,
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

In [21]:
# Reshaping the input to work with a basic RNN
tmp_x3 = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x3 = tmp_x3.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_x.shape)

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x3.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(bd_rnn_model.summary())
bd_rnn_model.fit(tmp_x3, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

(124074, 21, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 21, 1000)          2008000   
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           345345    
Total params: 2,353,345.0
Trainable params: 2,353,345
Non-trainable params: 0.0
_________________________________________________________________
None
Train on 99259 samples, validate on 24815 samples
Epoch 1/10
99259/99259 [==============================] - 87s - loss: 2.0097 - acc: 0.5509 - val_loss: 1.3122 - val_acc: 0.6278
Epoch 2/10
99259/99259 [==============================] - 85s - loss: 1.2307 - acc: 0.6348 - val_loss: 1.1491 - val_acc: 0.6559
Epoch 3/10
99259/99259 [==============================] - 85s - loss: 1.1259 - acc: 0.6554 - val_loss: 1.0926 - val_acc: 0.6614
Epoch 4/10
99259/99259 [==============================] - 85s - loss:

In [23]:
#evaluate performance on the test dataset

# Reshaping the test input
tmp_x3_test = pad(preproc_english_sentences_test, max_french_sequence_length)
tmp_x3_test = tmp_x3_test.reshape((-1, preproc_french_sentences.shape[-2], 1))

result = bd_rnn_model.evaluate(tmp_x3_test,preproc_french_sentences_test, verbose=0)
print("Bidirectional model accuracy on test dataset: {0:.4f}".format(result[1]) )

Bidirectional model accuracy on test dataset: 0.6883


In [24]:
#make 5 random predictions
make_prediction(5, bd_rnn_model)

Test 1
	English sentence: 	california is sometimes beautiful during fall but it is usually warm in february
	Prediction:     	californie france parfois parfois en juin mais mais il est généralement en en
	French sentence: 	californie est parfois belle à l'automne mais il est habituellement chaud en février
--------------------
Test 2
	English sentence: 	india is mild during january and it is sometimes busy in november
	Prediction:     	paris inde est en printemps et il est parfois parfois en juillet
	French sentence: 	l' inde est doux en janvier et il est parfois occupé en novembre
--------------------
Test 3
	English sentence: 	the lime is their least liked fruit but the apple is her least liked
	Prediction:     	la pomme est leur fruit moins aimé mais la raisin est son moins aimé
	French sentence: 	la chaux est leur moins aimé des fruits mais la pomme elle est moins aimé
--------------------
Test 4
	English sentence: 	new jersey is sometimes rainy during april and it is usually cold 

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [25]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    model = Sequential() 
    #model.add(LSTM(french_vocab_size,input_shape=input_shape[1:]))
    model.add(LSTM(500,input_shape=input_shape[1:]))
    model.add(RepeatVector(output_sequence_length))
    model.add(LSTM(500, return_sequences=True)) 
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    rmsprop =  RMSprop(lr=0.01)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=rmsprop,
                  metrics=['accuracy'])
    
    return model
tests.test_encdec_model(encdec_model)

In [26]:
# Reshaping the input to work with a basic RNN
tmp_x4 = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x4 = tmp_x4.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x4.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
print(encdec_rnn_model.summary())
encdec_rnn_model.fit(tmp_x4, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 500)               1004000   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 500)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 21, 500)           2002000   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 345)           172845    
Total params: 3,178,845.0
Trainable params: 3,178,845.0
Non-trainable params: 0.0
_________________________________________________________________
None
Train on 99259 samples, validate on 24815 samples
Epoch 1/10
99259/99259 [==============================] - 91s - loss: 3.6612 - acc: 0.3773 - val_loss: 3.2596 - val_acc: 0.4076
Epoch 2/10
99259/99259 [==============================] - 88s - loss: 3.187

In [27]:
#evaluate performance on the test dataset

# Reshaping the test input
tmp_x4_test = pad(preproc_english_sentences_test, max_french_sequence_length)
tmp_x4_test = tmp_x4_test.reshape((-1, preproc_french_sentences.shape[-2], 1))

result = encdec_rnn_model.evaluate(tmp_x4_test,preproc_french_sentences_test, verbose=0)
print("Encoder-decoder model accuracy on test dataset: {0:.4f}".format(result[1]) )

Encoder-decoder model accuracy on test dataset: 0.6211


In [28]:
#make 5 random predictions
make_prediction(5, encdec_rnn_model)

Test 1
	English sentence: 	india is never wonderful during january but it is sometimes cold in october
	Prediction:     	la france est parfois en en et il il est est en en
	French sentence: 	l' inde est jamais merveilleux en janvier mais il est parfois froid en octobre
--------------------
Test 2
	English sentence: 	the united states is sometimes mild during february and it is never dry in spring
	Prediction:     	les états unis est jamais froid en mois et il est il en en
	French sentence: 	les états unis est parfois doux en février et il est jamais sec au printemps
--------------------
Test 3
	English sentence: 	california is usually wet during june but it is never dry in april
	Prediction:     	la france est parfois en en et il il est est en en
	French sentence: 	californie est généralement humide en juin mais il est jamais sec en avril
--------------------
Test 4
	English sentence: 	california is usually freezing during december but it is sometimes chilly in autumn
	Prediction:     

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [38]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential() 
    model.add(Embedding(english_vocab_size, 300, input_length=input_shape[1]))
    model.add(Bidirectional(LSTM(250))) 
    model.add(RepeatVector(output_sequence_length))
    model.add(LSTM(500, return_sequences=True)) 
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    rmsprop =  RMSprop(lr=0.01)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=rmsprop,
                  metrics=['accuracy']) 
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [40]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
  
    # TODO: Train the neural network
    model = model_final(x.shape,max_french_sequence_length, english_vocab_size,french_vocab_size)
    print(model.summary())
    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 15, 300)           60000     
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 500)               1102000   
_________________________________________________________________
repeat_vector_14 (RepeatVect (None, 21, 500)           0         
_________________________________________________________________
lstm_37 (LSTM)               (None, 21, 500)           2002000   
_________________________________________________________________
time_distributed_18 (TimeDis (None, 21, 345)           172845    
Total params: 3,336,845.0
Trainable params: 3,336,845.0
Non-trainable params: 0.0
_________________________________________________________________
None
Train on 99259 samples, validate on 24815 samples
Epoch 1/20
99259/99259 [==============================] - 89s - loss: 3